## Pyspark Gutenberg project
Primero importaremos las librerias para crear una sesión de spark y obtener el libro desde la url.

In [16]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split, col, regexp_replace, size
import requests

Creamos la sesión para manejar los datos en pyspark

In [17]:
spark = SparkSession.builder.appName("Analaislibro").getOrCreate()

Descargamos el libro desde Gutenberg, y lo almacenamos en una variable

In [18]:
url = "https://www.gutenberg.org/cache/epub/164/pg164.txt"
response = requests.get(url)
libro_texto = response.text

Guardamos el contenido en un archivo local.

In [19]:
with open("libro.txt", "w", encoding="utf-8") as file:
    file.write(libro_texto)

Cargamos el texto en un **RDD** (Resilient Distributed Dataset)

In [21]:
rdd = spark.sparkContext.textFile("libro.txt")

Eliminamos líneas vacías.

In [22]:
rdd = rdd.filter(lambda line: line.strip() != "")

Buscamos los capítulos y taremos los resultados a una lista.

In [23]:
capitulos = rdd.filter(lambda line: line.strip().startswith("CHAPTER")).collect()

Usamos **flatMap** para dividir el texto en parrafos, y lo convertimos en un **DataFrame**.

In [26]:
parrafos = rdd.flatMap(lambda line: line.split("\n\n"))
df_parrafos = parrafos.map(lambda p: (p,)).toDF(["parrafo"])

Eliminamos caracteres especiales, dejando solo letrar y reducimos los espacios.

In [27]:
df_parrafos = df_parrafos.withColumn("parrafo_limpio", regexp_replace(col("parrafo"), "[^a-zA-Z\s]", "").alias("parrafo_limpio"))
df_parrafos = df_parrafos.withColumn("parrafo_limpio", regexp_replace(col("parrafo_limpio"), "\s+", " ").alias("parrafo_limpio"))

Dividimos el párrafo en oraciones, las contamos, dividimos las palabras y contamos el número de palabras en cada parrafo.

In [29]:
df_parrafos = df_parrafos.withColumn("oraciones", split(col("parrafo_limpio"), "\."))
df_parrafos = df_parrafos.withColumn("num_oraciones", size(col("oraciones")))
df_parrafos = df_parrafos.withColumn("palabras", split(col("parrafo_limpio"), " "))
df_parrafos = df_parrafos.withColumn("num_palabras", size(col("palabras")))

Ahora visualizamos una vista previa de nuestro **DataFtrame**.

In [15]:
df_parrafos.show(100)

+--------------------+--------------------+--------------------+-------------+--------------------+------------+
|             parrafo|      parrafo_limpio|           oraciones|num_oraciones|            palabras|num_palabras|
+--------------------+--------------------+--------------------+-------------+--------------------+------------+
|The Project Guten...|The Project Guten...|[The Project Gute...|            1|[The, Project, Gu...|          11|
|This ebook is for...|This ebook is for...|[This ebook is fo...|            1|[This, ebook, is,...|          14|
|most other parts ...|most other parts ...|[most other parts...|            1|[most, other, par...|          14|
|whatsoever. You m...|whatsoever You ma...|[whatsoever You m...|            1|[whatsoever, You,...|          14|
|of the Project Gu...|of the Project Gu...|[of the Project G...|            1|[of, the, Project...|          11|
|at www.gutenberg....|at wwwgutenbergor...|[at wwwgutenbergo...|            1|[at, wwwgutenberg.